In [1]:
import sys
import xarray as xr
from xgcm import Grid
from xgcm.autogenerate import generate_grid_ds
import os
import glob
import numpy as np
import gsw
sys.path.append('/Users/ignasi/Desktop/Oceanography/IEO/projects/stratification_CMIP6')
from data_functions import *
from cmip6_reanalyisis_handler import *



In [6]:
# Function to calculate wind curl and append it to the same file
def calculate_and_append_curl(files, output_dir):
    for fl in files:
        try:
            
            output_file = os.path.join(output_dir, os.path.basename(fl).replace('.nc', '_curl.nc'))
        
        # Check if the output file already exists
            if os.path.exists(output_file):
                print(f"Skipping {fl}: Curl file already exists.")
                continue

            ds = xr.open_dataset(fl,chunks={'time': 12})
            
            # Check if the required variables are present
            if 'tauu' not in ds or 'tauv' not in ds:
                print(f"Skipping {fl}: Required variables not found.")
                continue

            # Generate grid dataset for xgcm
            print(f" processing file {fl}")
            lat_2d = np.tile(ds.lat.values[:, np.newaxis], (1, len(ds.lon.values)))
            ds_full = generate_grid_ds(ds, {'Y': 'lat', 'X': 'lon'})
            grid = Grid(ds_full, periodic=['X'])
            
            print('grid generated')

            # Calculate grid differences
            dlong = grid.diff(ds_full.lon, 'X', boundary='extend', fill_value=np.nan)
            dlatg = grid.diff(ds_full.lat, 'Y', boundary='extrapolate', fill_value=np.nan)
            dlong[0] = dlong[1]  # Correct the first row if necessary
            
            print('diff generated')

            # Calculate distances
            dx, dy = dll_dist(dlong, dlatg, ds.lon, ds.lat)
            
            
            print('dist generated')

            # Compute derivatives
            du_dy = grid.diff(ds.tauu, 'Y', boundary='fill', fill_value=np.nan) / dy
            dv_dx = grid.diff(ds.tauv, 'X') / dx
            
            print('gradient generated')

            # Align the lat and lon for difference results
            du_dy['lat_left'] = dv_dx['lat'].values
            dv_dx['lon_left'] = du_dy['lon'].values
            du_dy = np.squeeze(du_dy.isel(lat=0).drop('lat'))

            du_dy = du_dy.rename({'lat_left': 'lat'})
            dv_dx = dv_dx.rename({'lon_left': 'lon'})

            # Calculate curl: dVdx - dUdy
            curl = dv_dx - du_dy
            curl = curl.rename('curl')
            
            print('curl generated')

            curl.to_netcdf(output_file)
            print(f"Saved curl to {output_file}.")

        except Exception as e:
            print(f"Error processing {fl}: {e}")
            
            

def calculate_SSDflux(files, output_path):
    # Required input variables
    required_vars = ['sos', 'tos', 'hfds', 'evspsbl', 'pr']
    
    for file in files:
        print(f"Processing file: {file}")
        # Open the NetCDF file
        ds = xr.open_dataset(file, chunks={'time': 12})
        
        # Check if all required variables exist in the dataset
        missing_vars = [var for var in required_vars if var not in ds.variables]
        if missing_vars:
            print(f"Warning: Skipping file {file} due to missing variables: {', '.join(missing_vars)}")
            ds.close()
            continue
        
        # Extract necessary variables
        sos = ds['sos']  # Sea Surface Salinity (PSU)
        tos = ds['tos']  # Sea Surface Temperature (degC)
        hfds = ds['hfds']  # Heat flux (W/m^2)
        evspsbl = ds['evspsbl']/1000  # Evaporation (kg/m^2/s)
        pr = ds['pr']/1000  # Precipitation (kg/m^2/s)
        
        # Ensure units for sos are in g/kg
        SA = np.squeeze(sos)   # Convert PSU to g/kg assuming 35 PSU = 35 g/kg
        
        # Density of seawater (using surface pressure, 0 dbar)
        rho = gsw.density.rho(SA, tos, 0)
        
        # Thermal contribution (Ft)
        alpha = gsw.alpha(SA, tos, 0)  # Thermal expansion coefficient (1/degC)
        Ft = -alpha * (hfds / 3850)  # Thermal contribution to SSDflux
        
        # Salinity contribution (Fs)
        beta = gsw.beta(SA, tos, 0)  # Haline contraction coefficient (1/(g/kg))
        Fs = rho * beta * SA * ((evspsbl - pr) / (1 - (SA / 1000)))  # Haline contribution
        
        # Total SSDflux (Frho)
        Frho = Ft + Fs  # Total SSDflux
        print(Ft)
        # Create a new Dataset for output
        output_ds = xr.Dataset(
            {
                "Ft": (("time", "lat", "lon"), Ft.values, {"units": "kg/m^2/s", "description": "Thermal contribution to SSDflux"}),
                "Fs": (("time", "lat", "lon"), Fs.values, {"units": "kg/m^2/s", "description": "Salinity contribution to SSDflux"}),
                "Frho": (("time", "lat", "lon"), Frho.values, {"units": "kg/m^2/s", "description": "Total SSDflux"})
            },
            coords={
                "time": ds["time"],
                "lat": ds["lat"],
                "lon": ds["lon"]
            },
            attrs={
                "description": "Thermal, salinity, and total SSDflux contributions computed from input variables."
            }
        )
        
        # Save the new dataset to the output directory
        output_filename = os.path.basename(file).replace('.nc', '_SSDflux.nc')
        output_filepath = os.path.join(output_path, output_filename)
        output_ds.to_netcdf(output_filepath)
        print(f"Saved SSDflux outputs to {output_filepath}")
        
        # Close datasets
        ds.close()
        output_ds.close()
        
        

def calculate_and_append_windstress_magnitude(files, output_dir):
    for fl in files:
        try:
            
            output_file = os.path.join(output_dir, os.path.basename(fl).replace('.nc', '_wsm.nc'))
        
        # Check if the output file already exists
            if os.path.exists(output_file):
                print(f"Skipping {fl}: Curl file already exists.")
                continue

            ds = xr.open_dataset(fl,chunks={'time': 12})
            
            # Check if the required variables are present
            if 'tauu' not in ds or 'tauv' not in ds:
                print(f"Skipping {fl}: Required variables not found.")
                continue

            wsm=np.sqrt(ds.tauu**2+ds.tauv**2)
            wsm = wsm.rename('wsm')
            
            print('wsm generated')

            wsm.to_netcdf(output_file)
            print(f"Saved curl to {output_file}.")

        except Exception as e:
            print(f"Error processing {fl}: {e}")


In [7]:
input_path = "/Volumes/Thalassa/CMIP6_forcings_RG_360"

output_path = "/Volumes/Thalassa/CMIP6_forcings_RG_360/wsm_output"
os.makedirs(output_path, exist_ok=True)

# List all NetCDF files in the directory
files = [os.path.join(input_path, f) for f in os.listdir(input_path) if f.endswith('.nc')]

calculate_and_append_windstress_magnitude(files, output_path)

wsm generated
Saved curl to /Volumes/Thalassa/CMIP6_forcings_RG_360/wsm_output/FGOALS-f3-L_atm_forcing_wsm.nc.
wsm generated
Saved curl to /Volumes/Thalassa/CMIP6_forcings_RG_360/wsm_output/E3SM-1-1-ECA_atm_forcing_wsm.nc.
wsm generated
Saved curl to /Volumes/Thalassa/CMIP6_forcings_RG_360/wsm_output/MIROC6_atm_forcing_wsm.nc.
wsm generated
Saved curl to /Volumes/Thalassa/CMIP6_forcings_RG_360/wsm_output/BCC-ESM1_atm_forcing_wsm.nc.
wsm generated
Saved curl to /Volumes/Thalassa/CMIP6_forcings_RG_360/wsm_output/ACCESS-CM2_atm_forcing_wsm.nc.
wsm generated
Saved curl to /Volumes/Thalassa/CMIP6_forcings_RG_360/wsm_output/SAM0-UNICON_atm_forcing_wsm.nc.
wsm generated
Saved curl to /Volumes/Thalassa/CMIP6_forcings_RG_360/wsm_output/NorESM2-LM_atm_forcing_wsm.nc.
wsm generated
Saved curl to /Volumes/Thalassa/CMIP6_forcings_RG_360/wsm_output/CanESM5_atm_forcing_wsm.nc.
wsm generated
Saved curl to /Volumes/Thalassa/CMIP6_forcings_RG_360/wsm_output/E3SM-1-0_atm_forcing_wsm.nc.
wsm generated
Sav

In [12]:
input_path = "/Volumes/Thalassa/CMIP6_forcings_RG_360"

output_path = "/Volumes/Thalassa/CMIP6_forcings_RG_360/curl_output"
os.makedirs(output_path, exist_ok=True)

# List all NetCDF files in the directory
files = [os.path.join(input_path, f) for f in os.listdir(input_path) if f.endswith('.nc')]

calculate_and_append_curl(files, output_path)

 processing file /Volumes/Thalassa/CMIP6_forcings_RG_360/FGOALS-f3-L_atm_forcing.nc
grid generated
diff generated
dist generated
gradient generated
curl generated
Saved curl to /Volumes/Thalassa/CMIP6_forcings_RG_360/curl_output/FGOALS-f3-L_atm_forcing_curl.nc.
 processing file /Volumes/Thalassa/CMIP6_forcings_RG_360/E3SM-1-1-ECA_atm_forcing.nc
grid generated
diff generated
dist generated
gradient generated
curl generated
Saved curl to /Volumes/Thalassa/CMIP6_forcings_RG_360/curl_output/E3SM-1-1-ECA_atm_forcing_curl.nc.
 processing file /Volumes/Thalassa/CMIP6_forcings_RG_360/MIROC6_atm_forcing.nc
grid generated
diff generated
dist generated
gradient generated
curl generated
Saved curl to /Volumes/Thalassa/CMIP6_forcings_RG_360/curl_output/MIROC6_atm_forcing_curl.nc.
 processing file /Volumes/Thalassa/CMIP6_forcings_RG_360/BCC-ESM1_atm_forcing.nc
grid generated
diff generated
dist generated
gradient generated
curl generated
Saved curl to /Volumes/Thalassa/CMIP6_forcings_RG_360/curl_out

In [3]:
#Calculate ssd flux
#formula:

#alpha=gsw.alpha(sos, tos, 0)
#beta=gsw.beta(sos, tos, 0) # 1/g Kg-1

#Ft=-alpha*(Q0/3850) 
#Fs=rho*beta*SA*((E-prep)/(1-(SA/1000)))

#Frho=Ft-Fs


#sos=psu
#tos=degC
#Q0=hfds=W m-2
#E=evspsbl= kg m-2 s-1 (downward)
#prep=pr=kg m-2 s-1


# Paths
input_path = "/Volumes/Thalassa/CMIP6_forcings_RG_360"
output_path = "/Volumes/Thalassa/CMIP6_forcings_RG_360/SSDflux_output"
os.makedirs(output_path, exist_ok=True)

# List all NetCDF files in the directory
files = [os.path.join(input_path, f) for f in os.listdir(input_path) if f.endswith('.nc')]

# Perform the SSDflux calculation
calculate_SSDflux(files, output_path)


Processing file: /Volumes/Thalassa/CMIP6_forcings_RG_360/FGOALS-f3-L_atm_forcing.nc
<xarray.DataArray (time: 540, lat: 180, lon: 360)>
dask.array<mul, shape=(540, 180, 360), dtype=float64, chunksize=(12, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1970-01-16T12:00:00 ... 2014-12-16T12:00:00
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
Saved SSDflux outputs to /Volumes/Thalassa/CMIP6_forcings_RG_360/SSDflux_output/FGOALS-f3-L_atm_forcing_SSDflux.nc
Processing file: /Volumes/Thalassa/CMIP6_forcings_RG_360/E3SM-1-1-ECA_atm_forcing.nc
<xarray.DataArray (time: 540, lat: 180, lon: 360)>
dask.array<mul, shape=(540, 180, 360), dtype=float64, chunksize=(12, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1970-01-16T12:00:00 ... 2014-12-16T12:00:00
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 .